# Computing Graphs and initial conditions

This notebook will instatiate and save the necessities for conducting the dynamical analysis.
Firstly the dataset will be elaborated and then graphs and initial conditions will be computed.
Might be necessary to change the data path (`PATH_TO_DATA`) or the dataset filnames (`WORD_VECTORS_FILENAME` and `ARTCLES_DF_FILENAME`).

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import networkx as nx
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

## These following have to be customized
PATH_TO_DATA = Path('../data')
# for joblib multithreading
N_THREADS = -1
# hyperparameters of the graph, get them from empirical data analysis
MIN_SIM = 0.75
TIME_THRESHOLD = 9*24 # in hours
TIME_MIN = 4 # in hours

These methods, used for building the graph, have been extracted and adapted from [this repo](https://github.com/elisamussumeci/modeling-news-spread).

In [28]:
def get_index(
    data, # DataFrame containing articles info
    timestamp, # time of current article
    similarities, # similarities of the current article
    max_dt, # max dt (in hours) for two articles to be linked
    min_dt, # min dt (in hours) for two articles to be linked
    min_similarity, # minimum cos sim distance for two articles to be linked
    outs, # current list of excluded articles
    ):
    while True:
        # get the maximum similarity w.r.t. older articles
        similarity = max(similarities)
        # get the index of such max similar article
        index = similarities.index(similarity)
        # get dt in terms of hours
        dt = (timestamp - data['timestamp'][index]).total_seconds() / 3600# / 24
        # similarity threshold
        if similarity < min_similarity:
            # return None to add index to outs
            return None
        # continue if article is in outs or its distant in time
        elif index in outs or dt > max_dt or dt <= min_dt:
            similarities[index] = 0
        # pass condition
        else:
            return index

def create_graph(
    dists_triu, # similarity matrix
    data, # DataFrame containing articles info
    time_max: int = 168, # max dt (in hours) for two articles to be linked
    time_min: int = 6, # min dt (in hours) for two articles to be linked
    sim_min: float = 0.8, # minimum cos sim distance for two articles to be linked
    ):
    # max number of nodes
    n_articles = dists_triu.shape[0]
    # instantiate the directed graph
    G = nx.DiGraph()
    # adding the first node
    G.add_node(0, #step=0,
               timestamp=data['timestamp'][0],
               source=data['source'][0],
               id=data['article_id'][0],
               #children=[],
              )
    # instatiating elimination list
    outs = []
    # loop on the other articles
    for i in range(1, n_articles):
        # get time of current article
        pub_i = data['timestamp'][i]
        # get similarities
        column = list(dists_triu[:, i])
        # get index of an article related to the current one
        index = get_index(data, pub_i, column, time_max, time_min, sim_min, outs)
        # if a relation was found
        if index != None:
            # if the related article has not already been inserted, insert it
            if index not in G.nodes():
                G.add_nodes_from([(index, {
                    'timestamp': data['timestamp'][index],
                    'source': data['source'][index],
                    'id': data['article_id'][index],
                })])
            # if the current article has not already been inserted, insert it
            if i not in G.nodes():
                G.add_nodes_from([(i, {
                    'timestamp': data['timestamp'][i],
                    'source': data['source'][i],
                    'id': data['article_id'][i]
                })])
            # linking the nodes
            G.add_edge(index, i)
        # if a relation wa not found
        else:
            # add current article to elimination listS
            outs.append(i)
    # return the graph
    return G

# TODO: describe method, add comments
def create_matrix_domain(graph):
    
    domain_list = []
    for_domain = tqdm(
        graph.nodes(),
        leave=True,
        unit='nodes_for_domain',
    )
    for pos in for_domain:
        node = graph.nodes()._nodes[pos]
        d = node['source']
        if d not in domain_list:
            domain_list.append(d)

    df = pd.DataFrame(0, index = domain_list, columns = domain_list)
    
    for_nodes = tqdm(
        graph.nodes(),
        leave=True,
        unit='nodes',
    )
    
    for pos in for_nodes:
        node = graph.nodes()._nodes[pos]
        d = node['source']
        successors = graph.successors(pos)
        for suc in successors:
            df[d][graph.nodes()._nodes[suc]['source']] += 1

    return [domain_list, df]

# TODO: describe method, add comments
def create_complete_adjacency(graph, matrix):
    
    for_cols = tqdm(
        [matrix[graph.nodes()._nodes[col]['source']] for col in graph.nodes()],
        leave=True,
        unit='nodes',
    )
    
    def get_node_probs(domain_col_probs):
        return np.array([
            domain_col_probs[graph.nodes()._nodes[row]['source']] for row in graph.nodes()
        ])
    
    probs = Parallel(n_jobs=N_THREADS)(delayed(get_node_probs)(i) for i in for_cols)
    df = pd.DataFrame(graph_complete, index=G.nodes(), columns=G.nodes())
    return df

Methods for initializing the dynamic simulation and creating the initial state.

In [3]:
def create_first_pubs(original_graph):
    dates_list = [original_graph.nodes()._nodes[node]['timestamp'] for node in original_graph.nodes()]
    fs = []
    print(min(dates_list).date())
    for node in original_graph.nodes():
        if original_graph.nodes()._nodes[node]['timestamp'].date() == min(dates_list).date():
            fs.append(original_graph.nodes()._nodes[node]['source'])
    return fs


def create_i0(list_first_pubs, domains):
    i0 = np.zeros(len(domains))
    for pos, i in enumerate(i0):
        if domains[pos] in list_first_pubs:
            i0[pos] = 1
            list_first_pubs.remove(domains[pos])

    return i0

Load data.

In [4]:
files = [
    'dists_triu.csv',
    'info_df.csv',
    'empirical_graph.gpickle',
    'empirical_graph_nodes.csv',
    'graph_original_domains_each_node.txt',
    'graph_complete.csv',
    'i0.csv',
    ]
stories = [
    'world_russia',
    'world_norway',
    'world_capitol_hill',
]
story_to_elaborate = 0
with open(PATH_TO_DATA/stories[story_to_elaborate]/files[1]) as csv_file:
    # dropping autospawned 'Unnamed: 0' column, and unecessary (since they are ordered already) 'article_id' column
    info_df = pd.read_csv(csv_file).drop(['Unnamed: 0'], 1)
info_df = info_df.rename(columns={'id': 'article_id'})
info_df['timestamp'] = pd.to_datetime(info_df.timestamp)
print(info_df.head())
with open(PATH_TO_DATA/stories[story_to_elaborate]/files[0]) as csv_file:
    # dropping autospawned 'Unnamed: 0' column, and unecessary (since they are ordered already) 'article_id' column
    dists_triu = pd.read_csv(csv_file, sep=',', header=None)
dists_triu = dists_triu.values
print(dists_triu.shape)

/tmp/ipykernel_67676/3622524216.py:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  info_df = pd.read_csv(csv_file).drop(['Unnamed: 0'], 1)


   index  article_id           timestamp  source
0      0  1926391701 2021-05-10 00:00:00  278722
1      0  1926430980 2021-05-10 00:00:00   20270
2      0  1926386660 2021-05-10 00:01:35    1123
3      0  1926388426 2021-05-10 00:01:35   63091
4      0  1926416043 2021-05-10 00:58:55    1123
(3323, 3323)


Creating, saving and drawing the Graphs instatiated using _networkX_ library.

In [29]:
G = create_graph(dists_triu, info_df, time_max=TIME_THRESHOLD, time_min=TIME_MIN, sim_min=MIN_SIM)
nx.write_gpickle(G, PATH_TO_DATA/stories[story_to_elaborate]/files[2])
# nx.draw(G, with_labels=True, font_weight='bold')

In [30]:
pd.DataFrame(dict(G.nodes())).transpose().to_csv(PATH_TO_DATA/stories[story_to_elaborate]/files[3])
all_nodes_domains = []
for i in G.nodes():
    all_nodes_domains.append(G.nodes()._nodes[i]['source'])

with open(PATH_TO_DATA/stories[story_to_elaborate]/files[4], 'w') as file:
    for item in all_nodes_domains:
        file.write("%s\n" % item)

In [31]:
domain_list, domain_matrix = create_matrix_domain(G)
domain_matrix.head()

  0%|          | 0/2889 [00:00<?, ?nodes_for_domain/s]

  0%|          | 0/2889 [00:00<?, ?nodes/s]

,278722,63091,655773,305988,146378,300133,20088,34596,66979,133933,...,131661,34587,31448,327794,295736,34586,64472,116287,4451,382935
278722,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63091,4,42,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
655773,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305988,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146378,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
graph_complete = create_complete_adjacency(G, domain_matrix)
graph_complete.head()

  0%|          | 0/2889 [00:00<?, ?nodes/s]

,0,8,10,11,12,13,14,15,16,17,...,3313,3314,3315,3316,3317,3318,3319,3320,3321,3322
0,0,4,1,1,1,1,1,1,1,1,...,0,4,2,2,4,4,0,4,0,2
8,0,42,0,0,0,1,0,0,0,0,...,2,42,51,51,42,42,0,42,2,51
10,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,1,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
as_numpy = np.array(graph_complete)
np.fill_diagonal(as_numpy, 0)
np.savetxt(PATH_TO_DATA/stories[story_to_elaborate]/files[5], as_numpy, delimiter=',')

Retrieve and save initial conditions.

In [34]:
list_first_pubs = create_first_pubs(G)
print('Length of first day pubs is {}'.format(len(list_first_pubs)))
I0 = create_i0(list_first_pubs, all_nodes_domains)
np.savetxt(PATH_TO_DATA/stories[story_to_elaborate]/files[6], I0, delimiter=',')
print('The number of inital infected is {}'.format(np.sum(I0)))
print(I0)

2021-05-10
Length of first day pubs is 60
The number of inital infected is 60.0
[1. 1. 1. ... 0. 0. 0.]
